In [ ]:
from scipy.stats import binned_statistic
import numpy as np
import matplotlib.pyplot as plt
data = np.load("voronoi_decay_data.npy", allow_pickle=True)

all_distances = []
all_values = []

# ------------------------------------------------------------
# Gather all cell distances and non-voronoiness values
# ------------------------------------------------------------
for trial in data:
    centers = trial["cell_centers"]
    values = trial["nonvoronoiness"]
    perturbed_vertex = trial["perturbed_vertex"]

    distances = np.linalg.norm(centers - perturbed_vertex[None, :], axis=1)

    all_distances.extend(distances)
    all_values.extend(values)

all_distances = np.array(all_distances)
all_values = np.array(all_values)

# ------------------------------------------------------------
# Bin distances to compute radial averages
# ------------------------------------------------------------
n_bins = 100  # number of radial bins
r_min, r_max = all_distances.min(), all_distances.max()
bin_means, bin_edges, _ = binned_statistic(
    all_distances, all_values, statistic='mean', bins=n_bins, range=(r_min, r_max)
)

# Compute bin centers
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

# ------------------------------------------------------------
# Normalize the decay curve (max = 1)
# ------------------------------------------------------------
bin_means_norm = bin_means / np.nanmax(bin_means)

# ------------------------------------------------------------
# Plot normalized decay curve
# ------------------------------------------------------------
plt.figure(figsize=(6, 4))
plt.plot(bin_centers, bin_means_norm, '-', lw=2, color='royalblue')
plt.scatter(bin_centers, bin_means_norm, s=10, color='royalblue', alpha=0.7)

plt.xlabel("Distance from perturbed vertex")
plt.ylabel("Normalized non-Voronoiness")
plt.title("Normalized spatial decay of perturbation effect")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()
